In [6]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.charts import format_fig
import plotly.graph_objects as go

# PolicyEngine color palette
BLUE_PRIMARY = "#2C6496"
TEAL_ACCENT = "#39C6C0"
DARK_GRAY = "#616161"
GRAY = "#808080"

In [ ]:
reform = Reform.from_dict({
  "gov.irs.credits.ctc.phase_out.amount": {
    "2025-01-01.2100-12-31": 25
  },
  "gov.irs.credits.ctc.amount.arpa[0].amount": {
    "2025-01-01.2100-12-31": 4800
  },
  "gov.irs.credits.ctc.amount.arpa[1].amount": {
    "2025-01-01.2100-12-31": 4800
  },
  "gov.irs.credits.ctc.phase_out.arpa.amount": {
    "2025-01-01.2100-12-31": 25
  },
  "gov.contrib.ctc.minimum_refundable.in_effect": {
    "2025-01-01.2100-12-31": True
  },
  "gov.contrib.ctc.per_child_phase_in.in_effect": {
    "2025-01-01.2100-12-31": True
  },
  "gov.irs.credits.ctc.phase_out.arpa.in_effect": {
    "2025-01-01.2100-12-31": True
  },
  "gov.irs.credits.ctc.refundable.phase_in.rate": {
    "2025-01-01.2100-12-31": 0.2
  },
  "gov.contrib.ctc.per_child_phase_out.in_effect": {
    "2025-01-01.2100-12-31": True
  },
  "gov.irs.credits.ctc.phase_out.threshold.JOINT": {
    "2025-01-01.2100-12-31": 200000
  },
  "gov.irs.credits.ctc.refundable.individual_max": {
    "2025-01-01.2100-12-31": 4800
  },
  "gov.irs.credits.ctc.phase_out.threshold.SINGLE": {
    "2025-01-01.2100-12-31": 100000
  },
  "gov.irs.credits.ctc.phase_out.threshold.SEPARATE": {
    "2025-01-01.2100-12-31": 100000
  },
  "gov.contrib.ctc.per_child_phase_out.avoid_overlap": {
    "2025-01-01.2100-12-31": True
  },
  "gov.irs.credits.ctc.refundable.phase_in.threshold": {
    "2025-01-01.2100-12-31": 0
  },
  "gov.irs.credits.ctc.phase_out.arpa.threshold.JOINT": {
    "2025-01-01.2100-12-31": 35000
  },
  "gov.contrib.ctc.minimum_refundable.amount[0].amount": {
    "2025-01-01.2100-12-31": 2400
  },
  "gov.contrib.ctc.minimum_refundable.amount[1].amount": {
    "2025-01-01.2100-12-31": 2400
  },
  "gov.irs.credits.ctc.phase_out.arpa.threshold.SINGLE": {
    "2025-01-01.2100-12-31": 25000
  },
  "gov.irs.credits.ctc.phase_out.arpa.threshold.SEPARATE": {
    "2025-01-01.2100-12-31": 25000
  },
  "gov.irs.credits.ctc.phase_out.threshold.SURVIVING_SPOUSE": {
    "2025-01-01.2100-12-31": 100000
  },
  "gov.irs.credits.ctc.phase_out.threshold.HEAD_OF_HOUSEHOLD": {
    "2025-01-01.2100-12-31": 100000
  },
  "gov.irs.credits.ctc.phase_out.arpa.threshold.SURVIVING_SPOUSE": {
    "2025-01-01.2100-12-31": 25000
  },
  "gov.irs.credits.ctc.phase_out.arpa.threshold.HEAD_OF_HOUSEHOLD": {
    "2025-01-01.2100-12-31": 25000
  }
}, country_id="us")


In [8]:
def create_situation(num_children, has_spouse=False, child_age=10, state="TX"):
    """Create a flexible situation with variable number of children and optional spouse."""
    situation = {
        "people": {
            "you": {
                "age": {"2025": 40}
            }
        },
        "axes": [
            [
                {
                    "name": "employment_income",
                    "count": 800,
                    "min": 0,
                    "max": 400000
                }
            ]
        ]
    }
    
    # Build members list
    members = ["you"]
    marital_unit_members = ["you"]
    
    # Add spouse if present
    if has_spouse:
        situation["people"]["spouse"] = {
            "age": {"2025": 40}
        }
        members.append("spouse")
        marital_unit_members.append("spouse")
    
    marital_units = {
        "your marital unit": {
            "members": marital_unit_members
        }
    }
    
    # Add children dynamically
    for i in range(1, num_children + 1):
        child_id = f"child {i}"
        situation["people"][child_id] = {
            "age": {"2025": child_age}
        }
        members.append(child_id)
        
        # Each child needs their own marital unit
        marital_units[f"{child_id} marital unit"] = {
            "members": [child_id],
            "marital_unit_id": {"2025": i}
        }
    
    # Complete all entity groups with the members list
    situation["families"] = {"your family": {"members": members}}
    situation["spm_units"] = {"your household": {"members": members}}
    situation["tax_units"] = {"your tax unit": {"members": members}}
    situation["households"] = {
        "your household": {
            "members": members,
            "state_name": {"2025": state}
        }
    }
    situation["marital_units"] = marital_units
    
    return situation

In [9]:
# Define scenarios to simulate
scenarios = [
    {"num_children": 1, "has_spouse": False, "name": "1 Child (Single)", "color": BLUE_PRIMARY, "dash": "solid"},
    {"num_children": 1, "has_spouse": True, "name": "1 Child (Married)", "color": BLUE_PRIMARY, "dash": "dash"},
    {"num_children": 2, "has_spouse": False, "name": "2 Children (Single)", "color": TEAL_ACCENT, "dash": "solid"},
    {"num_children": 2, "has_spouse": True, "name": "2 Children (Married)", "color": TEAL_ACCENT, "dash": "dash"},
]

# Create simulations for all scenarios
simulations = {}
for scenario in scenarios:
    situation = create_situation(
        num_children=scenario["num_children"],
        has_spouse=scenario["has_spouse"]
    )
    simulations[scenario["name"]] = {
        "simulation": Simulation(reform=reform, situation=situation),
        "color": scenario["color"],
        "dash": scenario["dash"]
    }

In [10]:
# Create the Plotly figure
fig = go.Figure()

# Add trace for each scenario
for name, data in simulations.items():
    employment_income = data["simulation"].calculate("employment_income", map_to="household", period=2025)
    ctc_value = data["simulation"].calculate("refundable_ctc", map_to="household", period=2025)
    
    fig.add_trace(go.Scatter(
        x=employment_income,
        y=ctc_value,
        mode='lines',
        name=name,
        line=dict(color=data["color"], width=2, dash=data["dash"]),
        hovertemplate='Employment Income: $%{x:,.0f}<br>CTC Value: $%{y:,.0f}<extra></extra>'
    ))

fig.update_layout(
    title='Child Tax Credit Value by Employment Income',
    xaxis_title='Employment Income ($)',
    yaxis_title='CTC Value ($)',
    xaxis=dict(tickformat="$,.0f"),
    yaxis=dict(tickformat="$,.0f", rangemode='tozero'),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01,
        font=dict(size=10)
    ),
    height=600,
    width=900,
    hovermode='x unified',
    template='plotly_white'
)

# Apply PolicyEngine styling
fig = format_fig(fig)
fig.show()